In [ ]:
#imports
import psycopg2
import requests
import os
from datetime import datetime
from datetime import timedelta

In [ ]:
#functions
def passback_params_func(oauth_consumer_key, lis_result_sourcedid, lis_outcome_service_url=None):
  n = oauth_consumer_key.find(oauth_consumer_key)+len('oauth_consumer_key')+3
  oauth_consumer_key = oauth_consumer_key[n:]
  oauth_consumer_key = oauth_consumer_key.strip("'  ")

  n = lis_result_sourcedid.find(lis_result_sourcedid)+len('lis_result_sourcedid')+3
  lis_result_sourcedid = lis_result_sourcedid[n:]
  lis_result_sourcedid = lis_result_sourcedid.strip("' ")

  if lis_outcome_service_url:
    n = lis_outcome_service_url.find(":") + 1
    lis_outcome_service_url = lis_outcome_service_url[n:]
    lis_outcome_service_url = lis_outcome_service_url.strip("' ")

  return oauth_consumer_key, lis_result_sourcedid, lis_outcome_service_url

def inner_dict_func(user_id, oauth_consumer_key, lis_result_sourcedid,
                    lis_outcome_service_url, is_correct, attempt_type,
                    created_at):
  return {"user_id":user_id,
          "oauth_consumer_key":oauth_consumer_key,
          "lis_result_sourcedid":lis_result_sourcedid,
          "lis_outcome_service_url":lis_outcome_service_url,
          "is_correct":is_correct,
          "attempt_type":attempt_type,
          "created_at":created_at}

#check log-files and open new log-file
start_url_connect = datetime.now()
log_file_dir = "/content/drive/MyDrive/Анализ Данных/3 Python база/logs/" #GoogleDrive`s path
log_file_path = f"{log_file_dir}{datetime.now().date()}.txt" #GoogleDrive`s file for today

dates = ( datetime.now().date(),
         (datetime.now()-timedelta(days=1)).date(),
         (datetime.now()-timedelta(days=2)).date(),
         (datetime.now()-timedelta(days=3)).date() )
files = [ f"{log_file_dir}{dt}.txt" for dt in dates ]
for f in os.scandir(log_file_dir):
  if f.is_file() and (f.path not in files):
    os.remove(f.path)

log_file = open(log_file_path, 'w') #open file

#variable for connection with URL
client='...' #change it ...
client_key='...' #change it ...
start='2025-04-01 00:00:00.000000' #change it ...
end='2025-05-01 00:00:00.000000' #change it ...
api_url = f"...client={client}&client_key={client_key}&start={start}&end={end}" #change it ...

#variable for connection to PosgreSQL Cloud-DataBase - change it ...
HOST = '...'
PORT = ...
DATABASE = '...'
USER = '...'
PASSWORD = '...'

#getting data with URL
#to do: поменять таймзону
log_file.write(f"{datetime.now()} INFO начало скачивания данных\n") #write to file 1
r = requests.get(api_url)

end_url_connect = datetime.now()
log_file.write(f"{end_url_connect-start_url_connect} INFO время на подключение по URL\n") #write to file 2

#checking status_code
status = r.status_code
if status != 200:
  log_file.write(f"{status} WARNING статус код\n") #write to file 2.1

#data processing
res = r.json()

output = []
for step in res:
  passback_params = step['passback_params']
  if passback_params == None:
    oauth_consumer_key, lis_result_sourcedid, lis_outcome_service_url = None, None, None
    inner_dict = inner_dict_func(step["lti_user_id"],
                                 oauth_consumer_key,
                                 lis_result_sourcedid,
                                 lis_outcome_service_url,
                                 step["is_correct"],
                                 step["attempt_type"],
                                 step["created_at"])
    output.append(inner_dict)
    continue

  passback_params = passback_params.strip("{}").split(",")
  passback_params = [i.strip() for i in passback_params]

  try:
    oauth_consumer_key, lis_result_sourcedid, lis_outcome_service_url = passback_params
    oauth_consumer_key, lis_result_sourcedid, lis_outcome_service_url =\
    passback_params_func(oauth_consumer_key, lis_result_sourcedid, lis_outcome_service_url)
    inner_dict = inner_dict_func(step["lti_user_id"],
                                 oauth_consumer_key,
                                 lis_result_sourcedid,
                                 lis_outcome_service_url,
                                 step["is_correct"],
                                 step["attempt_type"],
                                 step["created_at"])
    output.append(inner_dict)

  except ValueError:
    oauth_consumer_key, lis_result_sourcedid = passback_params
    oauth_consumer_key, lis_result_sourcedid, lis_outcome_service_url =\
    passback_params_func(oauth_consumer_key, lis_result_sourcedid)
    inner_dict = inner_dict_func(step["lti_user_id"],
                                 oauth_consumer_key,
                                 lis_result_sourcedid,
                                 lis_outcome_service_url,
                                 step["is_correct"],
                                 step["attempt_type"],
                                 step["created_at"])
    output.append(inner_dict)

end_reparam = datetime.now()
log_file.write(f"{end_reparam-end_url_connect} INFO время на обработку данных\n") #write to file 3

#connecting to database
#to do: поменять таймзону
connection_to_db = datetime.now()
log_file.write(f"{connection_to_db} INFO подключение к базе данных\n") #write to file 4
connection = psycopg2.connect(
    database=DATABASE,
    user=USER,
    password=PASSWORD,
    host=HOST,
    port=PORT
)
cursor = connection.cursor()

query = """insert into TABLE values (%s, %s, %s, %s, %s, %s, %s)""" #change TABLE

for data in output:
  user_id = data['user_id']
  oauth_consumer_key = data['oauth_consumer_key']
  lis_result_sourcedid = data['lis_result_sourcedid']
  lis_outcome_service_url = data['lis_outcome_service_url']
  is_correct = data['is_correct']
  attempt_type = data['attempt_type']
  created_at = data['created_at']
  cursor.execute(query, (user_id, oauth_consumer_key, lis_result_sourcedid,\
                         lis_outcome_service_url, is_correct, attempt_type,\
                         attempt_type, created_at))

cursor.close()
connection.close()
#to do: поменять таймзону
close_connecting_to_db = datetime.now()
log_file.write(f"{close_connecting_to_db} INFO загрузка данных в базу данных завершена, подключение к базе данных закрыто\n") #write to file 5
log_file.write(f"{close_connecting_to_db-connection_to_db} INFO время на загрузку данных\n") #write to file 6

log_file.close() #close file

In [ ]:
#to do*: разобрать подключение по API по закрытому ключу.json
print("Всего попыток запустить код/решить:")
print(len(output))
#126354

print("\nиз их успешных решений:")
print(len([i for i in output if i["is_correct"] == 1]))
#24281

print("\nПо пользователям.")
print("количество уникальных пользователей, пытавшихся решить задание:")
print(len({i["user_id"] for i in output}))
# 1217

print("из них, количество правильно решивших хоть что-то:")
print(len({i["user_id"] for i in output if i["is_correct"] == 1}))
#994

print("\nПо попыткам решить.")
print("всего попыток решить задачу:")
print(len([i for i in output if i["attempt_type"] == "submit"]))
#72922

print("их них успешных:")
print(len([i for i in output if (i["attempt_type"] == "submit" and i["is_correct"] == 1)]))
#24281

Всего попыток запустить код/решить:
126354

из их успешных решений:
24281

По пользователям.
количество уникальных пользователей, пытавшихся решить задание:
1217
из них, количество правильно решивших хоть что-то:
1066

По попыткам решить.
всего попыток решить задачу:
72922
их них успешных:
24281


In [ ]:
#to do**